<a href="https://colab.research.google.com/github/Puckas/Pukas/blob/master/CNNAudioTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Data Mining & Machine Learning 2 Project
Semester 3, Postgraduate Diploma in Data Analytics

National College of Ireland

*   Andrew Bruce (x19166591@student.ncirl.ie)
*   Luke Phillips (x19139454@student.ncirl.ie)
*   Mark McGrane (x19140606@student.ncirl.ie)
*   Stephen McMullan (x19139497@student.ncirl.ie)


## Initialise the Spotify API

Credentials come from https://developer.spotify.com/

In [ ]:
import os

os.environ['SPOTIPY_CLIENT_ID'] = 'bc0989005427470fadba1808bba205f0'
os.environ['SPOTIPY_CLIENT_SECRET'] = '9f8e5561e67441fe9737a477b3f4bb00'

!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

## Define Configuration

In [ ]:
batch_size = 32
image_size = (256, 256)

# Create Audio and Image download directories

In [ ]:
audio_download_dir = '/tmp/audio'
image_download_dir = '/tmp/image'

import shutil

try:
    shutil.rmtree(audio_download_dir)
except OSError as e:
    pass

try:
    shutil.rmtree(image_download_dir)
except OSError as e:
    pass

os.mkdir(audio_download_dir)
os.mkdir(image_download_dir)

## Select the Playlists

In [ ]:
# This is the Playlist URI - go to Spotify -> Playlist -> ... (beside the PLAY button) -> Share -> Copy Spotify URI
playlists = {
  #  'happy' : [
   #       'spotify:playlist:37i9dQZF1DWVlYsZJXqdym', # Happy Pop Hits
    #      'spotify:playlist:37i9dQZF1DWSf2RDTDayIx', # Happy Beats
     #     'spotify:playlist:37i9dQZF1DX7KNKjOK0o75' # Have a great day!
      #  ],
   # 'sad' : [
    #      'spotify:playlist:37i9dQZF1DX7qK8ma5wgG1', # Sad Songs
     #     'spotify:playlist:37i9dQZF1DWZUAeYvs88zc', # Sad Bops
      #    'spotify:playlist:37i9dQZF1DX3YSRoSdA634' # Life Sucks
      # ]
      'Angry' : [
            'spotify:playlist:05IS5lam05yuOY4nYKXqDg',   # Angry Music 
            'spotify:playlist:7odNW9I5GqtIX6DucjlKT1' #, # Angry - Mad - Pissed Off
            'spotify:playlist:4hlIJdFr4yoyllKu1q0hMb' # Get in loser

     ],
     
      'Relaxed' : [
      'spotify:playlist:37i9dQZF1DX4E3UdUs7fUx', # Afternoon Acoustic
      'spotify:playlist:37i9dQZF1DX2UgsUIg75Vg', # Chilled R&B
      'spotify:playlist:37i9dQZF1DX2czWA9hqErK'  # Easy
      ]
       
}

num_classes = len(playlists)

## Generate Spectrogram function using LibROSA

In [ ]:
import librosa
import matplotlib.pyplot as plt
import requests
import numpy as np

def generate_spectrogram(audio_name, audio_url, mood):
    # Download the 30 sec MP3 sample
    receive = requests.get(audio_url)
    audio_filepath = (audio_download_dir + '/' + audio_name + '_audio').replace(' ', '_')
    with open(audio_filepath, 'wb') as f:
      f.write(receive.content)

    # Load the sample file
    x, sr = librosa.load(audio_filepath, mono=True)

    cmap = plt.get_cmap('inferno')
    plt.figure(figsize=(10,10))
    plt.specgram(x, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    
    # included to create blank images for one class, to test if the net can discern between spectrograms and blanks (it can)  
    if mood == 'Relaxed':
      plt.clf()
      plt.figure(figsize=(10,10))
      a = np.matrix('1;1')
      plt.imshow(a, cmap='hot')

  # Uncomment the next line if you want to preview the images
  #  plt.show()
    plt.savefig(image_download_dir + '/' + mood + '/' + audio_name + '.png')
    plt.close()

## Generate Image file datasets

In [ ]:
def populate_data(playlists_dict):
  happy_song_count = 0
  sad_song_count = 0
  for mood, playlist_uris in playlists_dict.items():
    os.mkdir(image_download_dir + '/' + mood)

    for playlist_uri in playlist_uris:
      playlist = sp.playlist(playlist_uri)
      print("%s - Playlist: %s\n" % (mood, playlist['name']))

      tracks = sp.playlist_tracks(playlist_uri)
      num_tracks = 0
      for i, item in enumerate(tracks['items']):
        if num_tracks >= batch_size:
          break

        track = item['track']
       # print("   %d %32.32s %s" % (i, track['artists'][0]['name'], track['name']))
      
        track_preview = track['preview_url']

        if track_preview is not None:
          try:
            if mood == 'Angry':
              happy_song_count += 1
            elif mood == 'Relaxed':
              sad_song_count +=1
            generate_spectrogram(track['name'], track_preview, mood)
            print(happy_song_count, sad_song_count)
          except Exception as e:
            print(e)
          finally:
            num_tracks += 1
        else:
          print('Skipping - no audio sample found')

populate_data(playlists)

Angry - Playlist: "Angry Music"

1 0
2 0
3 0
4 0
5 0
Skipping - no audio sample found
6 0
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
7 0
8 0
9 0
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
10 0
11 0
12 0
Skipping - no audio sample found
13 0
14 0
15 0
Skipping - no audio sample found
16 0
17 0
18 0
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
19 0
20 0
21 0
Skipping - no audio sample found
Skipping - no audio sample found
Skipping - no audio sample found
22 0
23 0
Skipping - no audio sample found
24 0
25 0
26 0
Skipping - no audio sample found
Skipping - no audio sample found
27 0
28 0
29 0
30 0
Skipping - no audio sample found
Skipping - no audio sample found
3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


64 20


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


64 21


...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

## Prepare the Train and Test sets using Keras preprocessing

Tensorflow nightly build is required for tf.keras.preprocessing.image_dataset_from_directory support



In [7]:
!pip install tf-nightly
import tensorflow as tf

print('Tensorflow version: ', tf.__version__)
print('Keras version: ', tf.keras.__version__)

validation_split = 0.3
random_seed = 42

print(os.listdir(image_download_dir))

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  image_download_dir,
  labels='inferred',
  label_mode='categorical',
  validation_split=validation_split,
  subset='training',
  seed=random_seed,
  image_size=image_size,
  batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  image_download_dir,
  labels='inferred',
  label_mode='categorical',
  validation_split=validation_split,
  subset='validation',
  seed=random_seed,
  image_size=image_size,
  batch_size=batch_size
)

print(train_ds)
print(val_ds)

     |████████████████████████████████| 324.8MB 54kB/s 
     |████████████████████████████████| 460kB 35.9MB/s 
     |████████████████████████████████| 6.7MB 37.3MB/s 
Tensorflow version:  2.4.0-dev20200727
Keras version:  2.4.0
['Relaxed', 'Angry']
Found 159 files belonging to 2 classes.
Using 112 files for training.
Found 159 files belonging to 2 classes.
Using 47 files for validation.
<BatchDataset shapes: ((None, 256, 256, 3), (None, 2)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 256, 256, 3), (None, 2)), types: (tf.float32, tf.float32)>


## Create the Convolutional Neural Network (based on LeNet-5) using Keras

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# RGB = 3 image channels
image_chans = 3
input_shape = image_size + (image_chans,)

model = Sequential()

model.add(Rescaling(1./255, input_shape=input_shape))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 508032)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               6

## Compile the Model

In [9]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='nadam', 
    metrics=['accuracy']
    )

## Fit the Model

In [10]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

epochs = 10
callbacks = []

model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds)


Epoch 1/10
4/4 [==============================] - 10s 2s/step - loss: 15.3975 - accuracy: 0.5799 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 3s 755ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 3s 759ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 3s 761ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 3s 737ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 3s 735ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 3s 750ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0

# References



1.   Spotify for Developers: https://developer.spotify.com/
2.   Spotipy: https://spotipy.readthedocs.io/
3.   Librosa: https://librosa.org/
4.   Tensorflow CNN tutorial: https://www.tensorflow.org/tutorials/images/cnn
5.   Deep Learning with TensorFlow, Keras, and PyTorch: https://github.com/jonkrohn/DLTFpT
6.   Image classification from scratch: https://keras.io/examples/vision/image_classification_from_scratch/
7.   Keras API reference: https://keras.io/api/

